In [1]:
# -*- coding: utf-8 -*-

import arcpy


class Toolbox:
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "Toolbox"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [Tool]


class Tool:
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Tool"
        self.description = ""

    def getParameterInfo(self):
        """Define the tool parameters."""
        params = None
        return params

    def isLicensed(self):
        """Set whether the tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter. This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return


In [2]:
class CountyExporter:
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "County Highway Exporter"
        self.description = "This tool exports a zipped file geodatabase of highway data"


In [ ]:
full_fc_path = arcpy.Parameter(
    displayName="Highways Feature Class",
    name="Highways_Feature_Class",
    datatype="GPFeatureLayer",
    parameterType="Required",
    direction="Input"
)

In [1]:
    def getParameterInfo(self):
        """Define the tool parameters."""
        # First parameter
        full_fc_path = arcpy.Parameter(
            displayName="Highways Feature Class",
            name="Highways_Feature_Class",
            datatype="GPFeatureLayer",
            parameterType="Required",
            direction="Input")

        # Second parameter
        output_folder = arcpy.Parameter(
            displayName="Output Folder",
            name="Output_Folder",
            datatype="DEFolder",
            parameterType="Required",
            direction="Input")


        # Third parameter
        county = arcpy.Parameter(
            displayName="County",
            name="County",
            datatype="GPString",
            parameterType="Required",
            direction="Input")
        
        # list of all parameters
        params = [full_fc_path, output_folder, county]
        return params

In [2]:
def zip_county_highways(full_fc_path, output_folder, county):
    '''
    This function creates a zip file containing a file geodatabase with 
    a feature class containing only the highways for a specific county.

    Parameters:
        full_fc_path (str): The full path to the input feature class
        output_folder (str): The path to the output folder
        county (str): The name of the county to extract highways for

    Returns:
        str: The path to the zipped file geodatabase
    '''
    
    # remove spaces from county name
    county_no_spaces = county.replace(" ", "_")
    
    # create a file geodatabase
    fgdb = arcpy.management.CreateFileGDB(
        out_folder_path = output_folder,
        out_name = f"{county_no_spaces}_Output"
    )

    # Create a feature class
    output_fc = arcpy.conversion.ExportFeatures(
        in_features = full_fc_path,
        out_features = os.path.join(fgdb[0], 
                                    f"{county_no_spaces}_Highways"),
        where_clause = f"NAMELSAD = '{county}'"
    )
    
    source_fgdb_path = pathlib.Path(fgdb[0])

    # The name of the folder to place the File Geodatabase in
    fgdb_folder_name = source_fgdb_path.stem
    # The location of the folder to place the File Geodatabase in
    fgdb_folder_location = source_fgdb_path.parent
    # The path to the folder to place the File Geodatabase in
    fgdb_folder_path = fgdb_folder_location.joinpath(fgdb_folder_name)
    # The path of our copied File Geodatabase
    fgdb_path = fgdb_folder_path.joinpath(source_fgdb_path.name)

    # copy the file geodatabase into a temp folder
    shutil.copytree(
        source_fgdb_path, 
        fgdb_path, 
        ignore=shutil.ignore_patterns('*.lock')
    )

    # Zip the File Geodatabase
    zipped_fgdb = shutil.make_archive(
        base_name=fgdb_folder_path,  # The name of the archive, not including the file extension
        format="zip",  # The archive format
        root_dir=fgdb_folder_path,  # The directory to archive
    ) 

    # delete the file geodatabase
    arcpy.management.Delete(fgdb)

    # remove the temp folder
    shutil.rmtree(fgdb_folder_path)
    
    # return the zip file path
    return zipped_fgdb

In [ ]:
highways_feature_class = parameters[0].valueAsText
output_folder = parameters[1].valueAsText
county = parameters[2].valueAsText

In [ ]:
zip_county_highways(
    highways_feature_class, 
    output_folder,
    county
)

In [4]:
    def execute(self, parameters, messages):
        """The source code of the tool."""
        
        highways_feature_class = parameters[0].valueAsText
        output_folder = parameters[1].valueAsText
        county = parameters[2].valueAsText

        zip_county_highways(
                highways_feature_class, 
                output_folder,
                county
            )
        return

In [6]:
class Toolbox:
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "Toolbox"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [Tool, CountyExporter]

In [7]:
# -*- coding: utf-8 -*-

import arcpy
import shutil
import pathlib
import os

def zip_county_highways(full_fc_path, output_folder, county):
    '''
    This function creates a zip file containing a file geodatabase with 
    a feature class containing only the highways for a specific county.

    Parameters:
        full_fc_path (str): The full path to the input feature class
        output_folder (str): The path to the output folder
        county (str): The name of the county to extract highways for

    Returns:
        str: The path to the zipped file geodatabase
    '''
    
    # remove spaces from county name
    county_no_spaces = county.replace(" ", "_")
    
    # create a file geodatabase
    fgdb = arcpy.management.CreateFileGDB(
        out_folder_path = output_folder,
        out_name = f"{county_no_spaces}_Output"
    )

    # Create a feature class
    output_fc = arcpy.conversion.ExportFeatures(
        in_features = full_fc_path,
        out_features = os.path.join(fgdb[0], 
                                    f"{county_no_spaces}_Highways"),
        where_clause = f"NAMELSAD = '{county}'"
    )
    
    source_fgdb_path = pathlib.Path(fgdb[0])

    # The name of the folder to place the File Geodatabase in
    fgdb_folder_name = source_fgdb_path.stem
    # The location of the folder to place the File Geodatabase in
    fgdb_folder_location = source_fgdb_path.parent
    # The path to the folder to place the File Geodatabase in
    fgdb_folder_path = fgdb_folder_location.joinpath(fgdb_folder_name)
    # The path of our copied File Geodatabase
    fgdb_path = fgdb_folder_path.joinpath(source_fgdb_path.name)

    # copy the file geodatabase into a temp folder
    shutil.copytree(
        source_fgdb_path, 
        fgdb_path, 
        ignore=shutil.ignore_patterns('*.lock')
    )

    # Zip the File Geodatabase
    zipped_fgdb = shutil.make_archive(
        base_name=fgdb_folder_path,  # The name of the archive, not including the file extension
        format="zip",  # The archive format
        root_dir=fgdb_folder_path,  # The directory to archive
    ) 

    # delete the file geodatabase
    arcpy.management.Delete(fgdb)

    # remove the temp folder
    shutil.rmtree(fgdb_folder_path)
    
    # return the zip file path
    return zipped_fgdb

class Toolbox:
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "Toolbox"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [Tool, CountyExporter]


class Tool:
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Tool"
        self.description = ""

    def getParameterInfo(self):
        """Define the tool parameters."""
        params = None
        return params

    def isLicensed(self):
        """Set whether the tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter. This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return


class CountyExporter:
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "County Highway Exporter"
        self.description = "This tool exports a zipped file geodatabase of highway data"

    def getParameterInfo(self):
        """Define the tool parameters."""
        # First parameter
        full_fc_path = arcpy.Parameter(
            displayName="Highways Feature Class",
            name="Highways_Feature_Class",
            datatype="GPFeatureLayer",
            parameterType="Required",
            direction="Input")

        # Second parameter
        output_folder = arcpy.Parameter(
            displayName="Output Folder",
            name="Output_Folder",
            datatype="DEFolder",
            parameterType="Required",
            direction="Input")


        # Third parameter
        county = arcpy.Parameter(
            displayName="County",
            name="County",
            datatype="GPString",
            parameterType="Required",
            direction="Input")
        
        params = [full_fc_path, output_folder, county]
        return params

    def isLicensed(self):
        """Set whether the tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter. This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        
        highways_feature_class = parameters[0].valueAsText
        output_folder = parameters[1].valueAsText
        county = parameters[2].valueAsText

        zip_county_highways(
                highways_feature_class, 
                output_folder,
                county
            )
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return


Copyright 2025 Esri